# ETL code for DASS database

In [1]:
#Setting dependencies

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

#Import the python driver for PostgreSQL
import psycopg2
from passwords import password

### Read the csv file created during pre-processing

In [2]:
df = pd.read_csv('dass.csv')

In [3]:
# check the dataframe to ensure it's as expected
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39737 entries, 0 to 39736
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         39737 non-null  object 
 1   tipi1           39737 non-null  int64  
 2   tipi2           39737 non-null  int64  
 3   tipi3           39737 non-null  int64  
 4   tipi4           39737 non-null  int64  
 5   tipi5           39737 non-null  int64  
 6   tipi6           39737 non-null  int64  
 7   tipi7           39737 non-null  int64  
 8   tipi8           39737 non-null  int64  
 9   tipi9           39737 non-null  int64  
 10  tipi10          39737 non-null  int64  
 11  education       39737 non-null  int64  
 12  urban           39737 non-null  int64  
 13  gender          39737 non-null  int64  
 14  engnat          39737 non-null  int64  
 15  age_group       39737 non-null  object 
 16  age             39737 non-null  float64
 17  hand            39737 non-null 

In [4]:
# print the head to see the data
df.head()

,country,tipi1,tipi2,tipi3,tipi4,tipi5,tipi6,tipi7,tipi8,tipi9,...,major,Depression,Depression_cat,Anxiety,Anxiety_cat,Stress,Stress_cat,dep_cond,anx_cond,str_cond
0,IN,1,5,7,7,7,7,7,5,1,...,No Degree,27,3,34,4,40,4,3 Severe,4 Extremely Severe,4 Extremely Severe
1,US,6,5,4,7,5,4,7,7,1,...,No Degree,24,3,17,3,27,3,3 Severe,3 Severe,3 Severe
2,PL,2,5,2,2,5,6,5,5,3,...,No Degree,39,4,12,2,17,1,4 Extremely Severe,2 Moderate,1 Mild
3,US,1,1,7,4,6,4,6,1,6,...,Biology,16,2,17,3,16,1,2 Moderate,3 Severe,1 Mild
4,MY,2,5,3,6,5,5,5,6,3,...,Psychology,32,4,40,4,29,3,4 Extremely Severe,4 Extremely Severe,3 Severe


Create keys for each planned table and write each table out to a separate csv file.

In [5]:
class_labels = LabelEncoder()

In [6]:
df['age_grp_id'] = class_labels.fit_transform(df['age_group'].values)

In [7]:
# create a function for making ids for the tables. A new column is added to the dataframe when this is called

def make_id(df):
    str_id = df.apply(lambda x: '_'.join(map(str, x)), axis=1)
    return pd.factorize(str_id)[0]

df['person_id'] = make_id(df)

Verify person_id is unique

In [8]:
df['person_id'].value_counts()

0        1
26494    1
26487    1
26488    1
26489    1
        ..
13247    1
13248    1
13249    1
13250    1
39736    1
Name: person_id, Length: 39737, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39737 entries, 0 to 39736
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         39737 non-null  object 
 1   tipi1           39737 non-null  int64  
 2   tipi2           39737 non-null  int64  
 3   tipi3           39737 non-null  int64  
 4   tipi4           39737 non-null  int64  
 5   tipi5           39737 non-null  int64  
 6   tipi6           39737 non-null  int64  
 7   tipi7           39737 non-null  int64  
 8   tipi8           39737 non-null  int64  
 9   tipi9           39737 non-null  int64  
 10  tipi10          39737 non-null  int64  
 11  education       39737 non-null  int64  
 12  urban           39737 non-null  int64  
 13  gender          39737 non-null  int64  
 14  engnat          39737 non-null  int64  
 15  age_group       39737 non-null  object 
 16  age             39737 non-null  float64
 17  hand            39737 non-null 

Use LabelEncoder to create loc id based on combination of data that will be part of the location info table.

In [10]:
df['loc_info'] = df['urban'].astype(str) + df['country'] + df['person_id'].astype(str)
df['loc_id'] = class_labels.fit_transform(df['loc_info'].values)


Call the make_id function to create a dass id, temper id, and educ_id

In [11]:
df['dass_id'] = make_id(df[['Depression', 'Anxiety', 'Stress', 'person_id']])

In [12]:
df['dass_id'].value_counts()

0        1
26494    1
26487    1
26488    1
26489    1
        ..
13247    1
13248    1
13249    1
13250    1
39736    1
Name: dass_id, Length: 39737, dtype: int64

In [13]:
df['temper_id'] = make_id(df[['tipi1', 'tipi2', 'tipi3', 'tipi4', 'tipi5', 'tipi6', 'tipi7', 'tipi8', 'tipi9','tipi10','person_id']])

In [14]:
df['temper_id'].value_counts()

0        1
26494    1
26487    1
26488    1
26489    1
        ..
13247    1
13248    1
13249    1
13250    1
39736    1
Name: temper_id, Length: 39737, dtype: int64

In [15]:
df['educ_id'] = make_id(df[['education', 'major', 'person_id']])

In [16]:
df['educ_id'].value_counts()

0        1
26494    1
26487    1
26488    1
26489    1
        ..
13247    1
13248    1
13249    1
13250    1
39736    1
Name: educ_id, Length: 39737, dtype: int64

Remove interim column now that the key was created

In [17]:
df.drop(['loc_info'], axis='columns', inplace=True)

In [18]:
person_df = df.filter(['person_id','gender','race', 'orientation', 'married', 'hand', 'familysize', 'engnat', 'age_grp_id'], axis=1)
age_grp_df = df.filter(['age_grp_id', 'age_group'], axis=1)
loc_info_df = df.filter(['loc_id', 'urban', 'country', 'person_id'], axis=1)
dass_info_df = df.filter(['dass_id', 'Depression', 'Depression_cat', 'Anxiety', 'Anxiety_cat', 'Stress', 'Stress_cat', 'person_id'], axis=1)
temper_info_df = df.filter(['temper_id', 'tipi1', 'tipi2', 'tipi3', 'tipi4', 'tipi5', 'tipi6', 'tipi7', 'tipi8', 'tipi9','tipi10','person_id'],axis=1)
educ_info_df = df.filter(['educ_id','education', 'major', 'person_id'], axis=1)

Check each one to make sure data looks good

In [19]:
person_df.head()

,person_id,gender,race,orientation,married,hand,familysize,engnat,age_grp_id
0,0,2,10,1,1,1,2,2,0
1,1,2,70,5,1,2,4,1,0
2,2,2,60,3,1,1,3,2,0
3,3,2,70,5,1,2,5,1,0
4,4,2,10,1,1,3,4,2,0


In [20]:
age_grp_df.head()

,age_grp_id,age_group
0,0,0-20
1,0,0-20
2,0,0-20
3,0,0-20
4,0,0-20


In [21]:
age_grp_df.value_counts()

age_grp_id  age_group
1           20-24        14766
0           0-20         13820
2           25-29         5439
3           30-34         2194
5           40-49         1321
4           35-39         1138
6           50-59          764
7           60 & over      295
dtype: int64

There are duplicates in the age_grp_df since it is a domain table - will be dropping the duplicates

In [22]:
age_grp_df=age_grp_df.drop_duplicates()

In [23]:
age_grp_df.value_counts()

age_grp_id  age_group
0           0-20         1
1           20-24        1
2           25-29        1
3           30-34        1
4           35-39        1
5           40-49        1
6           50-59        1
7           60 & over    1
dtype: int64

In [24]:
loc_info_df.head()

,loc_id,urban,country,person_id
0,24673,3,IN,0
1,36868,3,US,1
2,36113,3,PL,2
3,38555,3,US,3
4,16345,2,MY,4


In [25]:
dass_info_df.head()

,dass_id,Depression,Depression_cat,Anxiety,Anxiety_cat,Stress,Stress_cat,person_id
0,0,27,3,34,4,40,4,0
1,1,24,3,17,3,27,3,1
2,2,39,4,12,2,17,1,2
3,3,16,2,17,3,16,1,3
4,4,32,4,40,4,29,3,4


In [26]:
temper_info_df.head()

,temper_id,tipi1,tipi2,tipi3,tipi4,tipi5,tipi6,tipi7,tipi8,tipi9,tipi10,person_id
0,0,1,5,7,7,7,7,7,5,1,1,0
1,1,6,5,4,7,5,4,7,7,1,5,1
2,2,2,5,2,2,5,6,5,5,3,2,2
3,3,1,1,7,4,6,4,6,1,6,1,3
4,4,2,5,3,6,5,5,5,6,3,3,4


In [27]:
educ_info_df.head()

,educ_id,education,major,person_id
0,0,2,No Degree,0
1,1,2,No Degree,1
2,2,2,No Degree,2
3,3,1,Biology,3
4,4,3,Psychology,4


Write each of the new dataframes to csv files.

In [28]:
person_df.to_csv("person.csv",index=False)
age_grp_df.to_csv("age_grp.csv",index=False)
loc_info_df.to_csv("loc_info.csv",index=False)
dass_info_df.to_csv("dass_info.csv",index=False)
temper_info_df.to_csv("temper_info.csv",index=False)
educ_info_df.to_csv("educ_info.csv",index=False)

# Data Model

Begin the database work - create the database, then create tables and load them.

# Create database

In [29]:
import psycopg2

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

 

# Connect to PostgreSQL DBMS

con = psycopg2.connect(user = "postgres", password = password);

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

 

# Obtain a DB Cursor

cursor          = con.cursor();

name_Database   = "dass";

 

# Create table statement

sqlCreateDatabase = "create database "+name_Database+";"

 

# Create a table in PostgreSQL database

cursor.execute(sqlCreateDatabase);

## Create tables

In [30]:
def create_tables():
    """ Use create commands to add tables to dass database in postgres"""
    
    commands = (
        """
        CREATE TABLE age_grp(
        age_grp_id SERIAL PRIMARY KEY,
        age_group CHAR(30))
        """,
        """
        CREATE TABLE person_info(
        person_id SERIAL PRIMARY KEY,
        gender INTEGER,
        race INTEGER,
        orientation INTEGER,
        married INTEGER,
        hand INTEGER,
        familysize INTEGER,
        engnat INTEGER,
        age_grp_id SERIAL REFERENCES age_grp(age_grp_id))
        """,
        """
        CREATE TABLE location_info(
        loc_id SERIAL PRIMARY KEY,
        urban INTEGER, 
        country CHAR(2),
        person_id SERIAL REFERENCES person_info(person_id))
        """,
        """
        CREATE TABLE educ_info(
        educ_id SERIAL PRIMARY KEY,
        educ_level INTEGER, 
        major CHAR(40),
        person_id SERIAL REFERENCES person_info(person_id))
        """,
        """
        CREATE TABLE temper_info(
        temper_id SERIAL PRIMARY KEY,
        tipi1 INTEGER,
        tipi2 INTEGER,
        tipi3 INTEGER,
        tipi4 INTEGER,
        tipi5 INTEGER,
        tipi6 INTEGER,
        tipi7 INTEGER,
        tipi8 INTEGER,
        tipi9 INTEGER,
        tipi10 INTEGER,
        person_id SERIAL REFERENCES person_info(person_id))
        """,
        """
        CREATE TABLE dass_info(
        dass_id SERIAL PRIMARY KEY,
        dep_score INTEGER,
        dep_level INTEGER,
        anxiety_score INTEGER,
        anxiety_level INTEGER,
        stress_score INTEGER,
        stress_level INTEGER,
        person_id SERIAL REFERENCES person_info(person_id))
        """
        )
    
    
    #Create a connection credentials to the PostgreSQL database
    try:
        conn = psycopg2.connect(user = "postgres", password = password, database = "dass")
        print("Connection established")
        
        #Create a cursor connection object to a PostgreSQL instance 
        cur = conn.cursor()
        print("Cursor opened")
    
        for command in commands:
            cur.execute(command)
    
        print("Tables created")
    
        #Commit transaction and prints the result successfully
        conn.commit()
        print ("Commit successful")

    #Handle the error throws by the command that is useful when using python while working with PostgreSQL
    except(Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL database", error)
        conn = None

    #Close the database connection
    finally:
        if(conn != None):
            cur.close()
            conn.close()
            print("PostgreSQL cursor & connection is now closed")
            
if __name__ == '__main__':
    create_tables()

Connection established
Cursor opened
Tables created
Commit successful
PostgreSQL cursor & connection is now closed


## Load tables

In [31]:
def load_table_func(conn,csv_file,table_na):
    cur = conn.cursor()
    print("Cursor opened for processing csv ",csv_file)
    
    with open(csv_file,'r') as i:
        # Skip the header row
        next(i)
        
        # copy the table
        cur.copy_from(i,table_na,sep=',',null='')
    

In [32]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dass")
print("Connection established")

load_table_func(conn,'age_grp.csv','age_grp')
load_table_func(conn,'person.csv','person_info')
load_table_func(conn,'loc_info.csv','location_info')
load_table_func(conn,'educ_info.csv','educ_info')
load_table_func(conn,'temper_info.csv','temper_info')
load_table_func(conn,'dass_info.csv','dass_info')
conn.commit()
conn.close()

Connection established
Cursor opened for processing csv  age_grp.csv
Cursor opened for processing csv  person.csv
Cursor opened for processing csv  loc_info.csv
Cursor opened for processing csv  educ_info.csv
Cursor opened for processing csv  temper_info.csv
Cursor opened for processing csv  dass_info.csv
